## Download HCP diffusion data

In [1]:
import os

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os.path as op
import os
%matplotlib inline

### Get the data from AWS S3

We assume that you have a file '~/.aws/credentials', that includes a section: 

    [hcp]
    AWS_ACCESS_KEY_ID=XXXXXXXXXXXXXXXX
    AWS_SECRET_ACCESS_KEY=XXXXXXXXXXXXXXXX
with the credentials you got [from HCP](https://wiki.humanconnectome.org/display/PublicData/How+To+Connect+to+Connectome+Data+via+AWS)

In [3]:
subjects = [992774, 994273]
#subjects = [991267]

In [4]:
base_dir = op.join(op.expanduser('~'), 'AFQ_data')
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
data_files = {}
for subject in subjects:
    sub_dir = op.join(base_dir, '%s'%subject)
    if not os.path.exists(op.join(base_dir, sub_dir)):
        os.mkdir(sub_dir)
        os.mkdir(os.path.join(sub_dir, 'sess'))
        os.mkdir(os.path.join(sub_dir, 'sess', 'dwi'))
        os.mkdir(os.path.join(sub_dir, 'sess', 'anat'))
    data_files[op.join(sub_dir, 'sess', 'dwi', 'dwi.bvals')] = 'HCP/%s/T1w/Diffusion/bvals'%subject
    data_files[op.join(sub_dir, 'sess', 'dwi', 'dwi.bvecs')] = 'HCP/%s/T1w/Diffusion/bvecs'%subject
    data_files[op.join(sub_dir, 'sess', 'dwi', 'dwi.nii.gz')] = 'HCP/%s/T1w/Diffusion/data.nii.gz'%subject
    data_files[op.join(sub_dir, 'sess', 'anat', 'T1w_acpc_dc.nii.gz')] = 'HCP/%s/T1w/T1w_acpc_dc.nii.gz'%subject

In [5]:
data_files

{'/Users/arokem/AFQ_data/992774/sess/anat/T1w_acpc_dc.nii.gz': 'HCP/992774/T1w/T1w_acpc_dc.nii.gz',
 '/Users/arokem/AFQ_data/992774/sess/dwi/dwi.bvals': 'HCP/992774/T1w/Diffusion/bvals',
 '/Users/arokem/AFQ_data/992774/sess/dwi/dwi.bvecs': 'HCP/992774/T1w/Diffusion/bvecs',
 '/Users/arokem/AFQ_data/992774/sess/dwi/dwi.nii.gz': 'HCP/992774/T1w/Diffusion/data.nii.gz',
 '/Users/arokem/AFQ_data/994273/sess/anat/T1w_acpc_dc.nii.gz': 'HCP/994273/T1w/T1w_acpc_dc.nii.gz',
 '/Users/arokem/AFQ_data/994273/sess/dwi/dwi.bvals': 'HCP/994273/T1w/Diffusion/bvals',
 '/Users/arokem/AFQ_data/994273/sess/dwi/dwi.bvecs': 'HCP/994273/T1w/Diffusion/bvecs',
 '/Users/arokem/AFQ_data/994273/sess/dwi/dwi.nii.gz': 'HCP/994273/T1w/Diffusion/data.nii.gz'}

In [ ]:
import botocore.session
import boto3
boto3.setup_default_session(profile_name='hcp')
s3 = boto3.resource('s3')
bucket = s3.Bucket('hcp-openaccess')

In [ ]:
for k in data_files.keys():
    if not op.exists(k):
        bucket.download_file(data_files[k], k)